# Transfer Learning
Ide utama dari transfer learning adalah sebagai berikut:

Kita menggunakan model yang sudah dibuat oleh orang lain untuk permasalahan lain yang lebih spesifik. Atau dengan kata lain, kita mengembangkan model yang telah dibuat sebelumnya agar memiliki kemampuan yang lebih spesifik sesuai dengan kebutuhan kita.

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
from torchvision import datasets, models, transforms
import time
import os
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device", device)

device cpu


# Transformasi Data

- Menginisiasi nilai mean dan standar deviasi untuk digunakan dalam data_transforms
- Transformasi untuk data training terdiri atas:
    - Resize dan Random Crop
    - Horizontal Flip
    - Konversi ke tensor
    - Normalisasi
- Transformasi untuk data testing terdiri atas:
    - Resize
    - Center Crop
    - Mengkonversi ke tensor, dan
    - Normalisasi

In [25]:
mean    = np.array([0.485, 0.456, 0.406])
std     = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

# Import Data
- https://github.com/mctosima/pengantarDLpytorch/blob/main/IPYNB%20Code/16_TransferLearning.ipynb
- Dataset dapat diunduh dari tautan berikut ini
- Dataset tersebut berisi gambar lebah dan semut yang telah dibagi kedalam dua folder untuk training dan validasi

In [26]:
data_directory  = './37_data/hymenoptera_data'
sets            = ['train', 'val']

image_datasets  = {}
dataloaders     = {}
dataset_sizes   = {}

for x in sets:
    image_datasets[x]   = datasets.ImageFolder(os.path.join(data_directory, x),data_transforms[x])
    dataloaders[x]      = torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)
    dataset_sizes[x]    = len(image_datasets[x])

class_names = image_datasets['train'].classes
print(class_names)

['ants', 'bees']


### Fungsi Training

In [27]:
def train_model(model, loss_fn, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss     = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                with torch.set_grad_enabled(phase == 'train'):
                    outputs  = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss     = loss_fn(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                
                running_loss     += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds==labels.data)
            
            if phase == 'train':
                scheduler.step()
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc  = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')
        print()
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60}m {time_elapsed % 60}s')

    return model

# Memuat Pretrained Model

- Kita menggunakan pretrained model yaitu resnet18
- Untuk menggunakan model ini, kita harus mengimport library torchvision.models
- Untuk mengetahui banyaknya fitur pada layer terakhir sebelum fully connected layer, kita dapat menggunakan model.fc.in_features
- Untuk mengubah fully-connected layer dari pre-trained model, kita gunakan model.fc = nn.Linear(in_features, num_classes)
- num_classes kita isi dengan 2 karena kelas yang akan diprediksi berupa lebah atau semut

In [28]:
model = models.resnet18(pretrained=True)

num_features = model.fc.in_features
print(f'Banyaknya Fitur: {num_features}')

model.fc = nn.Linear(num_features, 2)
model.to(device)

Banyaknya Fitur: 512


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Perintah model.fc = nn.Linear(in_features, num_classes) bertujuan untuk mengganti layer fc yang ada pada variabel model dengan layer yang baru yaitu nn.Linear(in_features, num_classes). Untuk melihat layer-layer yang terdapat pada sebuah model, dapat menggunakan perintah print(model)

In [29]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Menentukan Loss dan Optimizer

In [30]:
loss_fn   = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

Menggunakan Scheduler Scheduler digunakan untuk memperbaharui learning rate setiap epoch. Scheduler ini dapat diakses pada modul(library) optim

#### Sekilas tentang StepLR

- StepLR adalah scheduler yang menggunakan Step sebagai parameter
- Untuk menggunakannya, terdapat beberapa parameter yang dibutuhkan:
    - step_size adalah banyaknya epoch yang akan dihitung sebagai step
    - gamma adalah parameter untuk mengubah learning rate
    - optimizer adalah parameter untuk optimizer yang akan digunakan
    - last_epoch adalah parameter untuk menentukan epoch yang akan dihitung sebagai step
- Misalnya pada contoh di bawah ini, kita menggunakan step_size = 7 dan gamma = 0.1. Artinya: setiap 7 epoch, learning rate akan menjadi 0.1 * learning_rate

Perlu diketahui, dalam modul ini scheduler tidak berpengaruh dalam proses training karena epoch yang digunakan kurang dari jumlah step. Jika ingin menggunakan scheduler, perbesar nilai epoch. (Nilai epoch dikecilkan mengingat dibutuhkan proses komputasi yang lebih sederhana)

In [31]:
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Mengupdate Model

In [32]:
model = train_model(model, loss_fn, optimizer, step_lr_scheduler, num_epochs=5)

Epoch 1/5
----------
train Loss: 0.6238, Acc: 0.6557
val Loss: 0.4525, Acc: 0.7908

Epoch 2/5
----------
train Loss: 0.5402, Acc: 0.7377
val Loss: 0.3629, Acc: 0.8627

Epoch 3/5
----------
train Loss: 0.5005, Acc: 0.7500
val Loss: 0.2991, Acc: 0.9150

Epoch 4/5
----------
train Loss: 0.4359, Acc: 0.7623
val Loss: 0.2632, Acc: 0.9216

Epoch 5/5
----------
train Loss: 0.3938, Acc: 0.8238
val Loss: 0.2309, Acc: 0.9346

Training complete in 6.0m 11.386150121688843s


## Model Alternatif

- Semua layer kecuali layer terakhir (fully-connected) akan tetap menggunakan parameter pre-trained dari resnet-18 (dengan cara menonaktifkan grad).
- Namun, grad pada fully-connected layer akan diaktifkan (dengan memanggil nn.linear baru yang secara default grad==True)
- Sehingga, layer terakhir akan belajar

In [33]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_features = model.fc.in_features
print(f'Banyaknya Fitur: {num_features}')

model.fc = nn.Linear(num_features, 2)
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model = train_model(model, loss_fn, optimizer, step_lr_scheduler, num_epochs=5)

Banyaknya Fitur: 512
Epoch 1/5
----------
train Loss: 0.8982, Acc: 0.6230
val Loss: 0.5073, Acc: 0.7255

Epoch 2/5
----------
train Loss: 0.6813, Acc: 0.6967
val Loss: 0.6417, Acc: 0.7059

Epoch 3/5
----------
train Loss: 0.5640, Acc: 0.7705
val Loss: 0.1429, Acc: 0.9542

Epoch 4/5
----------
train Loss: 0.7171, Acc: 0.7459
val Loss: 0.2707, Acc: 0.9216

Epoch 5/5
----------
train Loss: 0.4718, Acc: 0.8074
val Loss: 0.4698, Acc: 0.8301

Training complete in 1.0m 59.82596278190613s


# Mencoba Model Alternatif-2

In [34]:
model = models.mobilenet_v2(pretrained=True)
print(model)

for param in model.parameters():
    param.requires_grad = False


num_features = 1280
model.classifier = nn.Sequential(
    nn.Linear(num_features, 2)
)

model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model = train_model(model, loss_fn, optimizer, step_lr_scheduler, num_epochs=5)

c:\Users\wilden\anaconda3\envs\pytorch-basic\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\wilden/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:11<00:00, 1.20MB/s]


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=